# 🧠 Fall Detection Model Training (LSTM)
시계열 포즈 데이터를 기반으로 낙상 감지 모델을 학습합니다.

In [ ]:
# ✅ 1. 필요 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

In [ ]:
# ✅ 2. 데이터셋 클래스 정의
class FallDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        for label_name in ['fall', 'normal']:
            label_dir = os.path.join(root_dir, label_name)
            label = 1 if label_name == 'fall' else 0
            for file in os.listdir(label_dir):
                if file.endswith('.npy'):
                    self.samples.append((os.path.join(label_dir, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        data = np.load(file_path)
        data = torch.tensor(data, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return data, label

In [ ]:
# ✅ 3. LSTM 분류 모델 정의
class FallLSTMClassifier(nn.Module):
    def __init__(self, input_size=54, hidden_size=128, num_layers=2):
        super(FallLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 2)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
# ✅ 4. 학습 루프
dataset = FallDataset('dataset')
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

model = FallLSTMClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    total_loss, total_acc = 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        total_acc += (pred == y).sum().item()

    acc = total_acc / len(dataset)
    print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}, Accuracy: {acc*100:.2f}%")

In [ ]:
# ✅ 5. 예측 테스트
model.eval()
sample, _ = dataset[0]
with torch.no_grad():
    output = model(sample.unsqueeze(0).to(device))
    pred = torch.argmax(output, dim=1)
    print('Fall Detected!' if pred.item() == 1 else 'Normal')